### Importing libraries and creating database

In [15]:
import pandas as pd
import sqlite3 
import os
import datetime

In [2]:
conn = sqlite3.connect('brEcommerce.db')
df = pd.read_csv(r"olist_customers_dataset.csv")
df.to_sql('customers', con = conn, if_exists = 'replace', index=False)

files_load = {
    'geolocation': 'olist_geolocation_dataset.csv',
    "order_items": "olist_order_items_dataset.csv",
    "order_payments": "olist_order_payments_dataset.csv",
    "orders": "olist_orders_dataset.csv",
    "products": "olist_products_dataset.csv",
    "sellers": "olist_sellers_dataset.csv",
    "product_category": "product_category_name_translation.csv" 
}

for table_n, csv_f in files_load.items():
    try:
        df = pd.read_csv(csv_f)
        df.to_sql(table_n,con = conn, if_exists = 'replace', index=False) 
    except Exception as e:
        print(f"Error {csv_f}: {e}")

conn.close()

### Joing tables

In [21]:
conn = sqlite3.connect('brEcommerce.db')
query = """
SELECT 
pc.product_category_name_english as Category,
SUM(oi.price) as total_revenue
FROM order_items oi 
JOIN products p ON oi.product_id = p.product_id 
JOIN product_category pc ON p.product_category_name = pc.product_category_name
GROUP BY product_category_name_english
ORDER BY total_revenue DESC
LIMIT 5;
"""

        
df_result = pd.read_sql(query, conn)
display(df_result)
conn.close()


,Category,total_revenue
0,health_beauty,1258681.34
1,watches_gifts,1205005.68
2,bed_bath_table,1036988.68
3,sports_leisure,988048.97
4,computers_accessories,911954.32


In [24]:
conn2 = sqlite3.connect('brEcommerce.db')
query2 = """
SELECT 
g.geolocation_lat as latitude, g.geolocation_lng as longitude
FROM orders o JOIN customers c ON o.customer_id = c.customer_id 
JOIN geolocation g ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
GROUP BY g.geolocation_lat, g.geolocation_lng
LIMIT 500;
"""

df_result2 = pd.read_sql(query2, conn2)
display(df_result2) 
conn2.close()

,latitude,longitude
0,-36.605374,-64.283946
1,-36.603837,-64.287433
2,-34.622400,-58.901888
3,-34.586422,-58.732101
4,-33.692616,-53.453972
...,...,...
495,-32.181839,-52.165585
496,-32.181753,-52.152247
497,-32.181613,-52.151316
498,-32.181549,-52.170984


In [25]:
conn3 = sqlite3.connect('brEcommerce.db')
query3 = """
SELECT strftime('%Y-%m', order_purchase_timestamp) AS month, 
    SUM(oi.price) AS total_revenue
FROM orders o JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY month
ORDER BY month
LIMIT 1000
"""

df_result3 = pd.read_sql(query3, conn3)
display(df_result3) 
conn3.close()

,month,total_revenue
0,2016-09,134.97
1,2016-10,40325.11
2,2016-12,10.90
3,2017-01,111798.36
4,2017-02,234223.40
5,2017-03,359198.85
6,2017-04,340669.68
7,2017-05,489338.25
8,2017-06,421923.37
9,2017-07,481604.52


In [ ]:
conn4 = sqlite3.connect('brEcommerce.db')
query4 = """
SELECT SUM(oi.price) AS current_revenue, SUM(oi.freight_value) AS current_freight
FROM orders o JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY current_revenue, current_freight
LIMIT 1000
"""

df_result4 = pd.read_sql(query4, conn4)
display(df_result4) 
conn4.close()